# Traffic-Revenue Statistical Analysis

This notebook analyzes the correlation between internal traffic data and revenue across the top 10/50/100/250 sites.

**Key insight:** Traffic values are monthly estimates at each snapshot, not daily visits. We test different snapshot selection methods to find the strongest correlation with revenue.

In [ ]:
# Imports
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Data Loading

In [ ]:
# Top 250 sites in ranked order (provided by user)
TOP_250_SITES = [
    'decoratoradvice.com', 'lookwhatmomfound.com', 'famousparenting.com', 'betterthisworld.com',
    'thestripesblog.com', 'etruesports.com', 'thenervousbreakdown.com', 'GFXMaker.com',
    'activepropertycare.com', 'playmyworld.com', 'bizwebgenius.com', 'redandwhitemagz.com',
    'myfavouriteplaces.org', 'voicesofconservation.org', 'mygreenbucks.net', 'beaconsoft.net',
    'traveltweaks.com', 'conversationswithgreg.com', 'thegamearchives.com', 'goodnever.com',
    'factom.com', 'revolvertech.com', 'hikhanacademy.org', 'lyncconf.com', 'thinkofgames.com',
    'goldengatemax.shop', 'aliensync.com', 'arcyart.com', 'mustardseedyear.com', 'disquantified.org',
    'harmonicode.com', 'termanchor.com', 'embedtree.com', 'whatutalkingboutwillis.com', 'smartsatta.com',
    'thechannelrace.org', 'bookspersonally.com', 'oneframework.net', 'projectrethink.org', 'gatorgross.com',
    'livingpristine.com', 'conversationswithbrittany.com', 'geekgadget.net', 'hearthstats.net',
    'coventchallenge.com', 'crypticstreet.com', 'freelogopng.com', 'wealthybyte.com',
    'amairaskincare.com.au', 'letsbuildup.org', 'doanphuongkimlien.com', 'luxuryinteriors.org',
    'whtvbox.com', 'mygardenandpatio.com', 'digitalrgs.org', 'masterrealtysolutions.com',
    'banlarbhumi.com', 'feathershort.com', 'fameblogs.net', 'middleclasshomes.net', 'icaiorg.net',
    'filmyzillah.com', 'logicalshout.com', 'playmastersclub.net', 'alternativeway.net',
    'mucicallydown.com', 'apartamentoscholloapartamentos.com', 'nationalpainreport.com',
    'w88w88hanoi.com', 'twiliaorg.com', 'thehake.com', 'setupseeker.xyz', 'playdedeus.com',
    'av19org.net', 'majinoukari.com', 'themeshgame.com', 'essec-kpmg.net', 'dhilisatta.com',
    'netzgames.net', 'eurogamersonline.com', 'harmoniclast.com', 'thinksano.com', 'jerseyexpress.net',
    'greediegoddess.com', 'iboomatelugu.com', 'dm-gaming.com', 'tech-biztech.com', 'besttarahi.com',
    'mazinoukari.com', 'etherions.com', 'zerodevice.net', 'thehometrotters.com', 'onthisveryspot.com',
    'g15tools.com', 'quantumcontactsshop.com', 'toptenlast.com', 'designmode24.com', 'aegaming.tv',
    'myinteriorpalace.com', 'bytesize-games.com', 'protongamer.com', 'bitnation-blog.com', 'dwchr.com',
    'onoxservices.com', 'musikcalldown.com', 'gamerawr.com', 'insidetheelevator.com', 'sportsfanfare.com',
    'satkamataka.com', 'buydomainspremium.com', 'freewayget.com', 'plugboxlinux.org', 'evolvedgross.com',
    'metsuiteorg.com', 'avstarnews.com', 'towersget.com', 'igxocosmetics.com', 'misumiskincare.com',
    'pawerbet.com', 'beerandmagic.com', 'fb88eu.net', 'bouncemediagroup.com', 'seismicpostshop.com',
    'venky12.com', 'theportablegamer.com', 'wibexeorg.com', 'songoftruth.org', 'reactcheck.com',
    'pushyourdesign.com', 'fudbollibre.com', 'elisehurstphotography.com', 'skylightvoice.com',
    'sports-report.net', 'conversationswithbianca.com', 'drhomey.com', 'quantumflooingservices.com',
    'areatsunami.com', 'cowded.com', 'entretech.org', 'investirebiz.com', 'mywirelesscoupons.com',
    'healthsciencesforum.com', 'nothing2hide.net', 'theboringmagazine.com', 'wolfcontactsshop.com',
    'wizzydigital.org', 'turbogeek.org', 'passionate-culinary-enterprises.com', 'triumphgross.com',
    'tubitymusic.com', 'ck2generator.com', 'checkerpointorg.com', 'northshoretimingonline.com',
    'sportsblitzzone.com', 'annoncetravesti.com', 'cookiesforlove.com', 'robthecoins.com',
    'tuple-tech.com', 'abithelp.com', 'saharahausa.com', 'homerocketrealty.com', 'val9jamusic.com',
    'alignlast.net', 'analysistheme.com', 'stayloosemusic.com', 'lotterygamedevelopers.com',
    'artsusshop.com', 'theguardianhib.com', 'greenheal.net', 'propagatenetworks.com',
    'thelowdownunder.com', 'bizboostpro.com', 'uptempomag.com', '21strongfoundation.org',
    'sunnylast.com', 'zigaero.com', 'android-underground.org', 'conversationswithjessica.com',
    'coststatus.com', 'pizzlemusic.com', 'thunderonthegulf.com', 'botbrobiz.com', 'blackrocklast.com',
    'epicgamerhq.com', 'prositesite.com', 'sportssavvyspot.com', 'cryptopronetwork.com',
    'safetyproductsmfg.com', 'formulagross.com', 'grosseasy.com', 'reality-movement.org',
    'feedbuzzard.com', 'leaguechannellife.net', 'Formotorbikes.com', 'theblockchainbrief.com',
    'liveamoment.org', 'sweedishlove.com', 'fightingforfutures.org', 'tech-bliss.com',
    'norstratiamrestaurant.com', 'digitalnewsalerts.com', 'trendsetti.com', 'mpgproworkstation.com',
    'toolmilk.com', 'springhillmedgroup.com', 'messgodess.com', 'leopardtheme.com', 'readlists.com',
    'eselmomentocv.com', 'shippingbellabeat.com', 'gtchgth.com', 'mydecine.com', 'aggreg8.net',
    'jordantrent.com', 'importantcool.com', 'tierraarea.com', 'crystalcreekland.com', 'alwaysthis.com',
    'playbattlesquare.com', 'durostech.com', 'beargryllsgear.org', 'fintechasia.net',
    'sportscentrehub.com', 'costofwar.com', 'boujeefitshapewear.com', 'letwomenspeak.com',
    'anywherestory.net', 'indianmatkamobi.com', 'technolotal.org', 'kdarchitects.net', 'snapsource.net',
    'americanlivewire.com', 'greenflourishhome.com', 'filmjila.com', 'thecurrentonline.net',
    'crazeforgamez.com', 'verdantnaturehome.com', 'thewritetrackpodcast.com', 'clearingdelight.com',
    'kalyanmatkachart.com', 'lotrizlotriz.com', 'gadgetsguruz.com', 'festivefitnessphilly.com',
    'terabytelabs.net', 'queenslot800.com', 'deephacks.org', 'kidsturncentral.com', 'anglospeed.com',
    'techsolutionsbiz.com', 'residencerenew.com'
]

# Create lowercase mapping for matching
TOP_250_LOWER = [s.lower() for s in TOP_250_SITES]

print(f"Total sites in ranking: {len(TOP_250_SITES)}")

In [ ]:
# Load revenue data
revenue_df = pd.read_csv('revenue-history.csv')
print(f"Revenue data shape: {revenue_df.shape}")
print(f"Columns: {revenue_df.columns.tolist()[:10]}... (showing first 10)")
revenue_df.head(3)

In [ ]:
# Load traffic data
traffic_df = pd.read_csv('traffic-data.csv')
print(f"Traffic data shape: {traffic_df.shape}")
print(f"Columns (first 10): {traffic_df.columns.tolist()[:10]}")
traffic_df.head(3)

In [ ]:
# Load DR history
dr_df = pd.read_csv('DR History.csv')
print(f"DR data shape: {dr_df.shape}")
dr_df.head(3)

## 2. Data Preprocessing

In [ ]:
def parse_revenue(val):
    """Parse revenue value from string format like '$1,234.56' to float"""
    if pd.isna(val) or val in ['-', 'x', '', ' ']:
        return np.nan
    try:
        # Handle string values
        if isinstance(val, str):
            val = val.strip().replace('$', '').replace(',', '')
            if val in ['-', 'x', '', ' ']:
                return np.nan
        return float(val)
    except (ValueError, TypeError):
        return np.nan

# Test the parser
test_vals = ['$1,234.56', '-', 'x', '$375.00', None, '']
print("Revenue parser test:")
for v in test_vals:
    print(f"  '{v}' -> {parse_revenue(v)}")

In [ ]:
# Process revenue data
# First row contains the date info, Website column has domain names
# Identify month columns (they contain years like 2022, 2023, etc.)

# Get the column that contains website names
website_col = 'Website'

# Identify month columns (format like 'Jan 2022', 'Feb 2022', etc.)
month_cols = [col for col in revenue_df.columns if any(year in str(col) for year in ['2022', '2023', '2024', '2025', '2026'])]
print(f"Found {len(month_cols)} month columns")
print(f"Sample month columns: {month_cols[:5]}")

# Filter to rows with valid website names
revenue_clean = revenue_df[revenue_df[website_col].notna() & (revenue_df[website_col] != '-')].copy()
print(f"\nRevenue rows with valid websites: {len(revenue_clean)}")

In [ ]:
# Create a clean revenue dataframe in long format
revenue_records = []

for idx, row in revenue_clean.iterrows():
    website = str(row[website_col]).strip().lower()
    niche = row.get('Niche', 'Unknown')
    
    for month_col in month_cols:
        rev_val = parse_revenue(row[month_col])
        if not pd.isna(rev_val):
            # Parse month column to date
            try:
                # Handle format like 'Jan 2022'
                month_date = pd.to_datetime(month_col, format='%b %Y')
                revenue_records.append({
                    'website': website,
                    'niche': niche,
                    'month': month_date,
                    'month_str': month_col,
                    'revenue': rev_val
                })
            except:
                pass

revenue_long = pd.DataFrame(revenue_records)
print(f"Revenue long format: {len(revenue_long)} records")
print(f"Unique websites: {revenue_long['website'].nunique()}")
print(f"Date range: {revenue_long['month'].min()} to {revenue_long['month'].max()}")
revenue_long.head()

In [ ]:
# Process traffic data
# Traffic data has Website column and date columns like 'Jul 1 - 2024'

# Get website column name (might be slightly different)
traffic_website_col = traffic_df.columns[1]  # Usually second column after index
print(f"Traffic website column: '{traffic_website_col}'")

# Identify date columns
traffic_date_cols = [col for col in traffic_df.columns if '-' in str(col) and any(year in str(col) for year in ['2024', '2025', '2026'])]
print(f"Found {len(traffic_date_cols)} date columns in traffic data")
print(f"Sample: {traffic_date_cols[:5]}")

In [ ]:
def parse_traffic_date(date_str):
    """Parse traffic date from format like 'Jul 1 - 2024' to datetime"""
    try:
        # Remove the ' - ' and parse
        clean = str(date_str).replace(' - ', ' ')
        return pd.to_datetime(clean, format='%b %d %Y')
    except:
        return None

# Test
test_dates = ['Jul 1 - 2024', 'Aug 15 - 2025', 'Jan 3 - 2026']
for d in test_dates:
    print(f"'{d}' -> {parse_traffic_date(d)}")

In [ ]:
# Create traffic long format
traffic_records = []

for idx, row in traffic_df.iterrows():
    website = str(row[traffic_website_col]).strip().lower() if pd.notna(row[traffic_website_col]) else None
    if not website:
        continue
        
    for date_col in traffic_date_cols:
        traffic_val = row[date_col]
        parsed_date = parse_traffic_date(date_col)
        
        if parsed_date and pd.notna(traffic_val):
            try:
                traffic_num = float(traffic_val)
                if traffic_num > 0:  # Only include positive values
                    traffic_records.append({
                        'website': website,
                        'date': parsed_date,
                        'traffic': traffic_num
                    })
            except (ValueError, TypeError):
                pass

traffic_long = pd.DataFrame(traffic_records)
print(f"Traffic long format: {len(traffic_long)} records")
print(f"Unique websites: {traffic_long['website'].nunique()}")
print(f"Date range: {traffic_long['date'].min()} to {traffic_long['date'].max()}")
traffic_long.head()

In [ ]:
# Filter to top 250 sites
revenue_top250 = revenue_long[revenue_long['website'].isin(TOP_250_LOWER)].copy()
traffic_top250 = traffic_long[traffic_long['website'].isin(TOP_250_LOWER)].copy()

print(f"Revenue records for top 250: {len(revenue_top250)} ({revenue_top250['website'].nunique()} sites)")
print(f"Traffic records for top 250: {len(traffic_top250)} ({traffic_top250['website'].nunique()} sites)")

# Check which sites are missing
sites_with_revenue = set(revenue_top250['website'].unique())
sites_with_traffic = set(traffic_top250['website'].unique())
sites_with_both = sites_with_revenue & sites_with_traffic

print(f"\nSites with both revenue AND traffic data: {len(sites_with_both)}")

## 3. Traffic Snapshot Selection Methods

Traffic values are **monthly estimates** at each snapshot. We test 4 methods to align with monthly revenue:
1. **Same-month latest** - Last snapshot within the revenue month
2. **Same-month average** - Average of snapshots within the month
3. **30-day lagged** - Snapshot from ~30 days before month end
4. **60-day lagged** - Snapshot from ~60 days before month end

In [ ]:
def get_month_end(month_date):
    """Get the last day of the month"""
    next_month = month_date + pd.offsets.MonthEnd(0)
    return next_month

def get_same_month_latest(traffic_df, website, month_date):
    """Get the latest traffic snapshot within the same month"""
    month_start = month_date.replace(day=1)
    month_end = get_month_end(month_date)
    
    mask = (traffic_df['website'] == website) & \
           (traffic_df['date'] >= month_start) & \
           (traffic_df['date'] <= month_end)
    
    subset = traffic_df[mask]
    if len(subset) == 0:
        return np.nan
    return subset.loc[subset['date'].idxmax(), 'traffic']

def get_same_month_avg(traffic_df, website, month_date):
    """Get average of all traffic snapshots within the same month"""
    month_start = month_date.replace(day=1)
    month_end = get_month_end(month_date)
    
    mask = (traffic_df['website'] == website) & \
           (traffic_df['date'] >= month_start) & \
           (traffic_df['date'] <= month_end)
    
    subset = traffic_df[mask]
    if len(subset) == 0:
        return np.nan
    return subset['traffic'].mean()

def get_lagged_traffic(traffic_df, website, month_date, lag_days):
    """Get traffic snapshot from approximately lag_days before month end"""
    month_end = get_month_end(month_date)
    target_date = month_end - pd.Timedelta(days=lag_days)
    
    # Get all snapshots for this website
    website_traffic = traffic_df[traffic_df['website'] == website]
    if len(website_traffic) == 0:
        return np.nan
    
    # Find closest snapshot to target date
    website_traffic = website_traffic.copy()
    website_traffic['date_diff'] = (website_traffic['date'] - target_date).abs()
    closest = website_traffic.loc[website_traffic['date_diff'].idxmin()]
    
    # Only use if within reasonable range (30 days of target)
    if closest['date_diff'].days > 30:
        return np.nan
    return closest['traffic']

# Test the functions
test_website = traffic_top250['website'].iloc[0]
test_month = pd.Timestamp('2024-10-01')
print(f"Testing with website: {test_website}, month: {test_month}")
print(f"Same-month latest: {get_same_month_latest(traffic_top250, test_website, test_month)}")
print(f"Same-month avg: {get_same_month_avg(traffic_top250, test_website, test_month)}")
print(f"30-day lagged: {get_lagged_traffic(traffic_top250, test_website, test_month, 30)}")
print(f"60-day lagged: {get_lagged_traffic(traffic_top250, test_website, test_month, 60)}")

In [ ]:
# Build combined dataset with all 4 traffic methods
# Only use months where we have traffic data (Jul 2024 onwards)

traffic_min_date = traffic_top250['date'].min()
print(f"Traffic data starts: {traffic_min_date}")

# Filter revenue to months with traffic data available
revenue_filtered = revenue_top250[revenue_top250['month'] >= traffic_min_date.replace(day=1)].copy()
print(f"Revenue records in traffic period: {len(revenue_filtered)}")

# Get unique website-month combinations
combined_records = []
total = len(revenue_filtered)

print("\nBuilding combined dataset (this may take a minute)...")
for i, (idx, row) in enumerate(revenue_filtered.iterrows()):
    if i % 500 == 0:
        print(f"  Processing {i}/{total}...")
    
    website = row['website']
    month = row['month']
    
    record = {
        'website': website,
        'month': month,
        'niche': row['niche'],
        'revenue': row['revenue'],
        'traffic_latest': get_same_month_latest(traffic_top250, website, month),
        'traffic_avg': get_same_month_avg(traffic_top250, website, month),
        'traffic_lag30': get_lagged_traffic(traffic_top250, website, month, 30),
        'traffic_lag60': get_lagged_traffic(traffic_top250, website, month, 60)
    }
    combined_records.append(record)

combined_df = pd.DataFrame(combined_records)
print(f"\nCombined dataset: {len(combined_df)} records")
combined_df.head(10)

In [ ]:
# Check data completeness
print("Data completeness:")
for col in ['traffic_latest', 'traffic_avg', 'traffic_lag30', 'traffic_lag60']:
    valid = combined_df[col].notna().sum()
    pct = 100 * valid / len(combined_df)
    print(f"  {col}: {valid} valid ({pct:.1f}%)")

## 4. Correlation Analysis by Segment

In [ ]:
def get_segment_sites(n):
    """Get top N sites from the ranking"""
    return [s.lower() for s in TOP_250_SITES[:n]]

def calc_correlation(df, traffic_col, revenue_col='revenue'):
    """Calculate Pearson and Spearman correlations with p-values"""
    # Drop rows with NaN in either column
    valid = df[[traffic_col, revenue_col]].dropna()
    
    if len(valid) < 3:
        return {'pearson_r': np.nan, 'pearson_p': np.nan, 
                'spearman_r': np.nan, 'spearman_p': np.nan, 'n': len(valid)}
    
    pearson_r, pearson_p = pearsonr(valid[traffic_col], valid[revenue_col])
    spearman_r, spearman_p = spearmanr(valid[traffic_col], valid[revenue_col])
    
    return {
        'pearson_r': pearson_r,
        'pearson_p': pearson_p,
        'spearman_r': spearman_r,
        'spearman_p': spearman_p,
        'n': len(valid)
    }

# Calculate correlations for each segment and method
segments = [10, 50, 100, 250]
traffic_methods = ['traffic_latest', 'traffic_avg', 'traffic_lag30', 'traffic_lag60']
method_labels = ['Same-Month Latest', 'Same-Month Avg', '30-Day Lagged', '60-Day Lagged']

results = []

for seg_size in segments:
    seg_sites = get_segment_sites(seg_size)
    seg_data = combined_df[combined_df['website'].isin(seg_sites)]
    
    for method, label in zip(traffic_methods, method_labels):
        corr = calc_correlation(seg_data, method)
        results.append({
            'Segment': f'Top {seg_size}',
            'Method': label,
            'Pearson r': corr['pearson_r'],
            'Pearson p': corr['pearson_p'],
            'Spearman r': corr['spearman_r'],
            'Spearman p': corr['spearman_p'],
            'N': corr['n']
        })

results_df = pd.DataFrame(results)
print("Correlation Results by Segment and Method:")
results_df

In [ ]:
# Create pivot table for easier comparison
pivot_pearson = results_df.pivot(index='Segment', columns='Method', values='Pearson r')
pivot_pearson = pivot_pearson[method_labels]  # Reorder columns

print("\n" + "="*70)
print("PEARSON CORRELATION (r) - Traffic vs Revenue")
print("="*70)
print(pivot_pearson.round(4).to_string())
print("\nHigher |r| = stronger correlation. Values closer to 1 or -1 are better.")

In [ ]:
# Visualize correlation comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Pearson heatmap
pivot_pearson_plot = results_df.pivot(index='Segment', columns='Method', values='Pearson r')
pivot_pearson_plot = pivot_pearson_plot[method_labels]
# Reorder rows
pivot_pearson_plot = pivot_pearson_plot.reindex(['Top 10', 'Top 50', 'Top 100', 'Top 250'])

sns.heatmap(pivot_pearson_plot, annot=True, fmt='.3f', cmap='RdYlGn', center=0, 
            ax=axes[0], vmin=-0.5, vmax=0.5)
axes[0].set_title('Pearson Correlation (r)\nTraffic vs Revenue', fontsize=12)
axes[0].set_xlabel('')

# Spearman heatmap
pivot_spearman_plot = results_df.pivot(index='Segment', columns='Method', values='Spearman r')
pivot_spearman_plot = pivot_spearman_plot[method_labels]
pivot_spearman_plot = pivot_spearman_plot.reindex(['Top 10', 'Top 50', 'Top 100', 'Top 250'])

sns.heatmap(pivot_spearman_plot, annot=True, fmt='.3f', cmap='RdYlGn', center=0,
            ax=axes[1], vmin=-0.5, vmax=0.5)
axes[1].set_title('Spearman Correlation (ρ)\nTraffic vs Revenue', fontsize=12)
axes[1].set_xlabel('')

plt.tight_layout()
plt.savefig('correlation_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Saved: correlation_heatmap.png")

In [ ]:
# Find the best performing method for each segment
print("\n" + "="*70)
print("BEST TRAFFIC METHOD BY SEGMENT (highest |Pearson r|)")
print("="*70)

for seg in ['Top 10', 'Top 50', 'Top 100', 'Top 250']:
    seg_results = results_df[results_df['Segment'] == seg]
    best_idx = seg_results['Pearson r'].abs().idxmax()
    best = seg_results.loc[best_idx]
    print(f"\n{seg}:")
    print(f"  Best method: {best['Method']}")
    print(f"  Pearson r = {best['Pearson r']:.4f} (p = {best['Pearson p']:.4f})")
    print(f"  Spearman r = {best['Spearman r']:.4f}")
    print(f"  Sample size: {best['N']:.0f}")

## 5. Revenue Efficiency Analysis

In [ ]:
# Calculate Revenue Per Traffic Unit (RPTU) for each site
# Using the best performing traffic method from above, or traffic_avg as default

# Aggregate by site
site_summary = combined_df.groupby('website').agg({
    'revenue': 'sum',
    'traffic_avg': 'mean',
    'traffic_latest': 'mean',
    'niche': 'first'
}).reset_index()

# Calculate RPTU (Revenue per Traffic Unit)
site_summary['rptu'] = site_summary['revenue'] / site_summary['traffic_avg']
site_summary['rptu'] = site_summary['rptu'].replace([np.inf, -np.inf], np.nan)

# Add ranking
site_summary['rank'] = site_summary['website'].apply(
    lambda x: TOP_250_LOWER.index(x) + 1 if x in TOP_250_LOWER else 999
)
site_summary = site_summary.sort_values('rank')

print(f"Sites with RPTU calculated: {site_summary['rptu'].notna().sum()}")
site_summary.head(10)

In [ ]:
# Top 10 most efficient sites (highest RPTU)
print("\n" + "="*70)
print("TOP 10 MOST EFFICIENT SITES (Highest Revenue per Traffic Unit)")
print("="*70)
top_efficient = site_summary.nlargest(10, 'rptu')[['rank', 'website', 'revenue', 'traffic_avg', 'rptu', 'niche']]
top_efficient['rptu'] = top_efficient['rptu'].round(2)
top_efficient['revenue'] = top_efficient['revenue'].apply(lambda x: f"${x:,.0f}")
top_efficient['traffic_avg'] = top_efficient['traffic_avg'].round(0)
print(top_efficient.to_string(index=False))

In [ ]:
# Bottom 10 least efficient sites (lowest RPTU) - potential monetization opportunities
print("\n" + "="*70)
print("BOTTOM 10 LEAST EFFICIENT SITES (Monetization Opportunities)")
print("="*70)
bottom_efficient = site_summary[site_summary['rptu'].notna()].nsmallest(10, 'rptu')[['rank', 'website', 'revenue', 'traffic_avg', 'rptu', 'niche']]
bottom_efficient['rptu'] = bottom_efficient['rptu'].round(2)
bottom_efficient['revenue'] = bottom_efficient['revenue'].apply(lambda x: f"${x:,.0f}")
bottom_efficient['traffic_avg'] = bottom_efficient['traffic_avg'].round(0)
print(bottom_efficient.to_string(index=False))

In [ ]:
# Scatter plot: Traffic vs Revenue with efficiency outliers highlighted
fig, ax = plt.subplots(figsize=(12, 8))

valid_sites = site_summary[site_summary['traffic_avg'].notna() & site_summary['revenue'].notna()]

# Calculate efficiency quartiles
q1 = valid_sites['rptu'].quantile(0.25)
q3 = valid_sites['rptu'].quantile(0.75)

# Color by efficiency
colors = []
for _, row in valid_sites.iterrows():
    if pd.isna(row['rptu']):
        colors.append('gray')
    elif row['rptu'] >= q3:
        colors.append('green')  # High efficiency
    elif row['rptu'] <= q1:
        colors.append('red')    # Low efficiency
    else:
        colors.append('blue')   # Average

scatter = ax.scatter(valid_sites['traffic_avg'], valid_sites['revenue'], 
                     c=colors, alpha=0.6, s=50)

ax.set_xlabel('Average Traffic (Monthly Estimate)', fontsize=12)
ax.set_ylabel('Total Revenue ($)', fontsize=12)
ax.set_title('Traffic vs Revenue by Site\nGreen=High Efficiency, Red=Low Efficiency (Opportunities)', fontsize=14)

# Add labels for top outliers
for _, row in valid_sites.nlargest(5, 'rptu').iterrows():
    ax.annotate(row['website'], (row['traffic_avg'], row['revenue']), fontsize=8)

plt.tight_layout()
plt.savefig('traffic_revenue_scatter.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Saved: traffic_revenue_scatter.png")

## 6. Additional Analyses

In [ ]:
# Niche Performance Analysis
print("\n" + "="*70)
print("CORRELATION BY NICHE")
print("="*70)

niche_results = []
for niche in combined_df['niche'].dropna().unique():
    if pd.isna(niche) or niche == 'Unknown':
        continue
    niche_data = combined_df[combined_df['niche'] == niche]
    if len(niche_data) < 10:
        continue
    
    corr = calc_correlation(niche_data, 'traffic_avg')
    niche_results.append({
        'Niche': niche[:40],  # Truncate long names
        'Sites': niche_data['website'].nunique(),
        'Records': corr['n'],
        'Pearson r': corr['pearson_r'],
        'Spearman r': corr['spearman_r']
    })

niche_df = pd.DataFrame(niche_results).sort_values('Pearson r', ascending=False)
print(niche_df.round(4).to_string(index=False))

In [ ]:
# Portfolio Concentration Analysis
print("\n" + "="*70)
print("PORTFOLIO CONCENTRATION")
print("="*70)

# Total revenue by site
total_revenue = site_summary['revenue'].sum()
site_summary_sorted = site_summary.sort_values('revenue', ascending=False)

for n in [10, 50, 100]:
    top_n_revenue = site_summary_sorted.head(n)['revenue'].sum()
    pct = 100 * top_n_revenue / total_revenue
    print(f"Top {n} sites: ${top_n_revenue:,.0f} ({pct:.1f}% of total)")

In [ ]:
# Seasonality Analysis
print("\n" + "="*70)
print("SEASONALITY ANALYSIS")
print("="*70)

combined_df['month_num'] = combined_df['month'].dt.month
combined_df['month_name'] = combined_df['month'].dt.strftime('%b')

monthly_avg = combined_df.groupby('month_num').agg({
    'revenue': 'mean',
    'traffic_avg': 'mean'
}).round(2)

month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
monthly_avg.index = [month_names[i-1] for i in monthly_avg.index]

print("\nAverage Revenue & Traffic by Month:")
print(monthly_avg.to_string())

In [ ]:
# Seasonality visualization
fig, ax1 = plt.subplots(figsize=(12, 5))

x = range(len(monthly_avg))
ax1.bar(x, monthly_avg['revenue'], alpha=0.7, color='green', label='Avg Revenue')
ax1.set_xlabel('Month')
ax1.set_ylabel('Average Revenue ($)', color='green')
ax1.tick_params(axis='y', labelcolor='green')
ax1.set_xticks(x)
ax1.set_xticklabels(monthly_avg.index)

ax2 = ax1.twinx()
ax2.plot(x, monthly_avg['traffic_avg'], color='orange', marker='o', linewidth=2, label='Avg Traffic')
ax2.set_ylabel('Average Traffic', color='orange')
ax2.tick_params(axis='y', labelcolor='orange')

plt.title('Seasonality: Average Revenue & Traffic by Month', fontsize=14)
fig.legend(loc='upper right', bbox_to_anchor=(0.9, 0.9))
plt.tight_layout()
plt.savefig('seasonality.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Saved: seasonality.png")

## 7. Summary & Key Findings

In [ ]:
print("\n" + "="*70)
print("EXECUTIVE SUMMARY")
print("="*70)

print("\n📊 DATA OVERVIEW:")
print(f"   • Sites analyzed: {combined_df['website'].nunique()}")
print(f"   • Date range: {combined_df['month'].min().strftime('%b %Y')} to {combined_df['month'].max().strftime('%b %Y')}")
print(f"   • Total revenue records: {len(combined_df)}")

print("\n📈 CORRELATION FINDINGS:")
# Find overall best method
best_overall = results_df.loc[results_df['Pearson r'].abs().idxmax()]
print(f"   • Strongest correlation found: {best_overall['Segment']} using {best_overall['Method']}")
print(f"   • Pearson r = {best_overall['Pearson r']:.4f}")

# Check if lagged methods perform better
lag30_avg = results_df[results_df['Method'] == '30-Day Lagged']['Pearson r'].abs().mean()
lag60_avg = results_df[results_df['Method'] == '60-Day Lagged']['Pearson r'].abs().mean()
same_month_avg = results_df[results_df['Method'] == 'Same-Month Latest']['Pearson r'].abs().mean()

if lag30_avg > same_month_avg or lag60_avg > same_month_avg:
    print(f"   • ✓ CONFIRMED: Lagged traffic shows stronger correlation than same-month")
    print(f"     (30-day avg: {lag30_avg:.4f}, 60-day avg: {lag60_avg:.4f} vs same-month: {same_month_avg:.4f})")
else:
    print(f"   • Same-month traffic shows similar or stronger correlation than lagged")

print("\n💰 EFFICIENCY INSIGHTS:")
if len(top_efficient) > 0:
    print(f"   • Most efficient site: {top_efficient.iloc[0]['website']}")
if len(bottom_efficient) > 0:
    print(f"   • Biggest monetization opportunity: {bottom_efficient.iloc[0]['website']}")

print("\n📁 FILES SAVED:")
print("   • correlation_heatmap.png")
print("   • traffic_revenue_scatter.png")
print("   • seasonality.png")